In [ ]:
from IPython.display import display, HTML
display(HTML('''
<style>
body {
    background-color: #1e1e1e !important;
    color: white !important;
}
.widget-label, .widget-html-content, .widget-button, .form-control, .widget-readout {
    color: white !important;
}
</style>
'''))

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import json
import os

# 저장 리스트
todo_items = []

# 레이아웃 설정
layout_input = widgets.Layout(width="100%")
layout_button = widgets.Layout(width="49%", height="36px")
layout_full = widgets.Layout(width="100%")
layout_progress = widgets.Layout(width="100%", height="12px")
layout_half = widgets.Layout(width="49%")

# 전체 진행률 바
progress_bar = widgets.FloatProgress(min=0, max=100, value=0, layout=layout_progress, bar_style="")
progress_label = widgets.Label(value="0%", layout=widgets.Layout(justify_content="flex-end"))

# 리스트 출력 박스
todo_box = widgets.VBox()

# 진행률 색상 함수
def progress_color(value):
    gray = int(255 - value * 2.55)
    return f"rgb({gray},{gray},{gray})"

# 전체 진행률 업데이트
def update_overall_progress():
    if not todo_items:
        progress_bar.value = 0
        progress_label.value = "0%"
        return
    total = sum(i["progress"] for i in todo_items) / len(todo_items)
    progress_bar.value = total
    progress_bar.style = {"bar_color": progress_color(total)}
    progress_label.value = f"{int(total)}%"

# 리스트 갱신 함수
def refresh_list():
    todo_box.children = []
    for idx, item in enumerate(todo_items, start=1):
        title = widgets.Label(f"# {idx}. {item['title']}", layout=layout_full)
        bar = widgets.FloatProgress(value=item["progress"], min=0, max=100, layout=layout_progress)
        bar.style = {"bar_color": progress_color(item["progress"])}
        slider = widgets.IntSlider(value=item["progress"], min=0, max=100, step=1, layout=layout_full)

        percent_label = widgets.Label(f"{item['progress']}%", layout=layout_full)

        date = widgets.Label(f"마감일 {item['due'].strftime('%Y-%m-%d')}" if item["due"] else "마감일 -", layout=layout_half)
        priority = widgets.Label(f"중요도 {item['priority']}", layout=layout_half)
        row4 = widgets.HBox([date, priority], layout=layout_full)

        complete = widgets.Button(description="완료✅", button_style="success", layout=layout_button)
        delete = widgets.Button(description="삭제❌", button_style="danger", layout=layout_button)
        row5 = widgets.HBox([complete, delete], layout=layout_full)

        def on_slider_change(change, i=item, bar=bar, label=percent_label):
            i["progress"] = change["new"]
            bar.value = change["new"]
            bar.style = {"bar_color": progress_color(change["new"])}
            label.value = f"{change['new']}%"
            update_overall_progress()

        def on_complete(b, i=item):
            i["progress"] = 100
            refresh_list()
            update_overall_progress()

        def on_delete(b, i=item):
            todo_items.remove(i)
            refresh_list()
            update_overall_progress()

        slider.observe(on_slider_change, names="value")
        complete.on_click(on_complete)
        delete.on_click(on_delete)

        card = widgets.VBox([title, bar, slider, percent_label, row4, row5],
                            layout=widgets.Layout(border="1px solid #ccc", padding="10px", margin="10px 0px"))
        todo_box.children += (card,)

    update_overall_progress()

# 입력 폼 위젯
title_input = widgets.Text(placeholder="할 일 입력", layout=layout_full)
progress_input = widgets.IntSlider(min=0, max=100, step=1, value=0, layout=layout_full)
due_input = widgets.DatePicker(layout=layout_full)
priority_input = widgets.Dropdown(options=["낮음", "보통", "높음"], value="보통", layout=layout_full)

add_btn = widgets.Button(description="추가", button_style="success", layout=layout_full)
save_btn = widgets.Button(description="저장", button_style="info", layout=layout_button)
load_btn = widgets.Button(description="불러오기", button_style="warning", layout=layout_button)

# 입력 폼 배치
input_widgets = widgets.VBox([
    title_input,
    progress_input,
    due_input,
    priority_input,
    add_btn,
    widgets.HBox([save_btn, load_btn], layout=layout_full),
    widgets.HTML("<b style='color:white; text-align:center;'>최종 진행률</b>"),
    widgets.HBox([progress_bar, progress_label], layout=layout_full),
])

def add_task(b):
    if not title_input.value.strip():
        return
    todo_items.append({
        "title": title_input.value.strip(),
        "progress": progress_input.value,
        "due": due_input.value,
        "priority": priority_input.value
    })
    title_input.value = ""
    progress_input.value = 0
    due_input.value = None
    priority_input.value = "보통"
    refresh_list()

def save_data(b):
    with open("todo_data.json", "w") as f:
        json.dump([{
            "title": i["title"],
            "progress": i["progress"],
            "due": i["due"].strftime('%Y-%m-%d') if i["due"] else None,
            "priority": i["priority"]
        } for i in todo_items], f)

def load_data(b):
    if not os.path.exists("todo_data.json"):
        return
    with open("todo_data.json", "r") as f:
        raw = json.load(f)
    todo_items.clear()
    for i in raw:
        todo_items.append({
            "title": i["title"],
            "progress": i["progress"],
            "due": datetime.strptime(i["due"], "%Y-%m-%d").date() if i["due"] else None,
            "priority": i["priority"]
        })
    refresh_list()

add_btn.on_click(add_task)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

# 전체 앱 출력
display(widgets.VBox([
    widgets.HTML("<h2 style='color:white;'>📁 DAVID mk34</h2>"),
    input_widgets,
    todo_box
], layout=widgets.Layout(background_color="#1e1e1e", padding="20px")))

refresh_list()
